# **San Francisco Crime Analysis & Prediction**




# Gliederung

<!-- 
    Frame the problem and look at the big picture.
    Get the data.
    Explore the data to gain insights.
    Prepare the data to better expose the underlying data patterns to Machine Learning algorithms.
    Explore many different models and short-list the best ones.
    Fine-tune your models and combine them into a great solution.
    Present your solution.
    Launch, monitor, and maintain your system.
-->

1. [Einleitung](#1-einleitung)
2. [Aufgabenstellung](#2-problemstellung)
3. [Import](#3-import)
4. [Datenbereinigung](#4-datenbereinigung)
5. [Exploration](#5-exploration)
6. [Vorbereitung](#6-vorbereitung)
7. [Modellierung](#7-modellierung)
8. [Ergebnis](#8-ergebnis)



# 1. Einleitung 

San Francisco war berüchtigt dafür, einige der weltweit bekanntesten Verbrecher auf der unentrinnbaren Insel Alcatraz unterzubringen. Heute ist die Stadt eher für ihre Technologieszene als für ihre kriminelle Vergangenheit bekannt. Ziel dieser Analyse ist eine Klassifizierung und Vorhersage von ausgewählten Verbrechenskategorien, basierend auf Zeit, Ort und weiteren Features. Als Grundlage hierfür dienen Kriminalberichte der letzten 14 Jahre, welche Daten aus allen Vierteln San Franciscos enthalten. 


# 2. Problemstellung

<!-- Frame the problem and look at the big picture. -->

Für die weitere Betrachtung werden im Folgenden die ersten Schritte unternommen, um eine Kategorie eines Verbrechens in San Francisco vorherzusagen. Um den Umfang der Daten einzuschränken beschränkt sich diese Analyse ausschließlich auf die nachstehenden Kategorien:


 *Larceny/Theft*, *Assault*, *Drug/Narcotic*, *Vehicle Theft* und *Burglary* 


Zur Vorhersage soll **eine** dieser Methoden verwendet werden: 

*Regression*, *Klassifikation* oder *Clustering*


Im Verlauf der Analyse wird die Entscheidung zur Vorhersage einer Kategorie eines Verbrechens auf die Klassifikation fallen.



# 3. Import

Im ersten Abschnitt dieser Analyse werden verschiedene Python-Bibliotheken importiert, um Datenanalyse, Visualisierung und maschinelles Lernen durchzuführen. Außerdem werden die entsprechenden Daten importiert.

In [ ]:
import json
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import seaborn as sns
colors = sns.color_palette(None, 5)
import matplotlib.pyplot as plt

import geopandas as gpd
import geoplot as gplt
from geopandas import GeoDataFrame
from geopandas.tools import sjoin
from shapely.geometry import Point
from shapely.geometry import Polygon, MultiPolygon

from meteostat import Point, Hourly

from itertools import product
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn import tree

from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# 4. Datenbereinigung 

Im nächsten Schritt folgt die Datenbereinigung, damit die Qualität der Daten überprüft werden kann. Außerdem werden Maßnahmen zur Bereinigung des Datensatzes durchgeführt.

In [ ]:
crime = pd.read_csv("data/train.csv")
crime["CrimeId"] = crime.index
crime.head()


Zunächst werden die ersten fünf Zeilen aus dem Datensatz ausgegeben, um ein erstes Gefühl für die Daten und Kriminalitätsinformationen zu bekommen.

In [ ]:
crime['Dates'] = pd.to_datetime(crime['Dates'])

print('First date: ', str(crime['Dates'].min()))
print('Last date: ', str(crime['Dates'].max()))
print('crime data shape: ', crime.shape)

## Selektion

Wie in der Problemstellung beschrieben, werden die Daten so gefiltert, dass nur *Larceny/Theft*, *Assault*, *Drug/Narcotic*, *Vehicle Theft* und *Burglary* als Kategorien vertreten sind.

In [ ]:
crime["Category"].unique()

In [ ]:
categories = [i.upper() for i in ["Larceny/Theft", "Assault", "Drug/Narcotic", "Vehicle Theft", "Burglary"]]

crime = crime.loc[crime["Category"].isin(categories)]

crime.head()

## Duplikate

Im nächsten Schritt werden die Duplikate im Datensatz untersucht:

In [ ]:
crime.duplicated().sum()

In [ ]:
crime.drop_duplicates(inplace=True)

## Datentypen

In [ ]:
crime.dtypes

Ein Blick auf die Datentypen zeigt, dass sie bereits in einem gänfigen Format vorliegen.
Lediglich die 'Dates' Spalte wird weiter unterteilt für spätere Analysen.

In [ ]:
crime["Dates"] = pd.to_datetime(crime["Dates"])
crime["YearMonth"] = crime['Dates'].dt.strftime('%Y-%m')
crime["YearMonthDay"] = crime['Dates'].dt.strftime('%Y-%m-%d')
crime["MonthDay"] = crime['Dates'].dt.strftime('%m-%d')
crime["Year"] = crime['Dates'].dt.strftime('%Y')
crime["Month"] = crime['Dates'].dt.strftime('%m')
crime["Day"] = crime['Dates'].dt.strftime('%d')
crime["TimeOfDay"] = crime['Dates'].dt.strftime('%H')

## Falsche Werte

Im nächsten Schritt wird betrachtet, ob die Koordinaten falsche Werte enthalten könnten.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import seaborn as sns

def create_gdf(df):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X, df.Y), crs='epsg:4326')
    return gdf


crime_gdf = create_gdf(crime)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

fig, ax = plt.subplots(figsize=(10, 10))

world.plot(ax=ax, color='white', edgecolor='black')
crime_gdf.plot(ax=ax, color='red', markersize=10)  # Adjust markersize as needed

ax.set_aspect('equal')

#sns.despine(ax=ax, left=True, right=True, top=True, bottom=True)

ax.set_xticks([])  # Use set_xticks to hide x-axis ticks
ax.set_yticks([])  # Use set_yticks to hide y-axis ticks


plt.show()


Ein Blick auf die Karte genügt um zu sehen, dass sich einige Punkte außerhalb der USA und außerhalb von San Fransisco befinden. Diese Punkte werden konkret lokalisiert und im nächsten Schritt ausgegeben. 

In [ ]:
print(crime_gdf.loc[crime_gdf.Y > 50].count()[0])
crime_gdf.loc[crime_gdf.Y > 50].sample(5)

Zusammengefasst gibt der Code also die Anzahl der Datensätze mit einer Y-Koordinate größer als 50 aus und zeigt dann fünf zufällige Datensätze mit dieser Bedingung an. Dies könnte darauf hindeuten, dass es im GeoDataFrame einige Datensätze mit ungewöhnlichen oder fehlerhaften geografischen Koordinaten gibt. Damit diese auffälligen Daten nicht aus der Analyse ausgeschlossen werden, können die Mittelwerte der vorhandenen Koordinaten der jeweiligen Polizeidistrikte genutzt werden.

In [ ]:
crime.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

imp = SimpleImputer(strategy='mean')

for district in crime['PdDistrict'].unique():
    crime.loc[crime['PdDistrict'] == district, ['X', 'Y']] = imp.fit_transform(
        crime.loc[crime['PdDistrict'] == district, ['X', 'Y']])

crime_gdf = create_gdf(crime)

Es wird folglich für jedes einzigartige Polizeidistrikt ('PdDistrict') im DataFrame crime der Imputer verwendet, um fehlende Werte in den Spalten 'X' und 'Y' durch den Mittelwert der vorhandenen Werte im jeweiligen Distrikt zu ersetzen.

## Fehlende Werte

In [ ]:
crime.isnull().sum()

In [ ]:
if sum(crime.isnull().any()*1):
    print("Es gibt fehlende Daten.")
else:
    print("Es gibt keine fehlenden Daten")

Nachdem die Koordinaten bereinigt wurden, werden weitere fehlende Werte berücksichtigt. Dafür wird die obenstehende Funktion verwendet um die Anzahl der fehlenden Werte in jedem Attribut (Spalte) des DataFrames zu zählen.

Wie aus der Tabelle abgelesen werden kann, gibt es für alle Kategorien keine fehlenden Werte. 




# 5. Exploration

Nachdem die Datenbereinigung abgeschlossen wurde, wird nun der Fokus auf die Exploration der Daten gelegt. Hierbei handelt es sich um einen Prozess, bei dem der Datensatz analysiert wird, um ein besseres Verständnis für die enthaltenen Variablen zu entwickeln. Ziel ist es, Muster, Trends oder ungewöhnliche Beobachtungen zu identifizieren. Dieser Prozess hilft bei der Vorbereitung der Daten für die weitere Analyse und Modellbildung.

## Deskriptive Statistik

### Balkendiagramme

#### Verbrechen je Bezirk

In [ ]:
data_cat = crime.groupby('Category').count().iloc[:, 0]. sort_values(ascending=False)
data = data_cat.reindex(np.append(np.delete(data_cat.index, 1), 'OTHER OFFENSES'))



In [ ]:
plt.figure(figsize=(3, 3))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        x = (data_cat.values / data_cat.values.sum()) * 100,
        y = data_cat.index,
        orient='h',
        palette="Blues_r")

plt.title('Incidents per Crime Category', fontdict={'fontsize': 16})
plt.xlabel('Incidents (%)')

plt.show()

Für einen ersten Überblick werden Einzelfälle pro Kategorie in Prozent aufgeschlüsselt. Es ist deutlich zu erkennen, dass 'Larceny/Theft' nahezu 50 Prozent der Fälle ausmacht. Dahingegen werden nur knapp unter 10 Prozent der Fälle als 'Burglary' klassifiziert.'

In [ ]:
data_cat.describe()

Für die weitere Einordnung wird die Funktion describe() genutzt. Wie bereits in der Selektion deutlich wurde, wird in der Analyse nur mit den fünf ausgewählten Kategorien gearbeitet. Außerdem können einige grundlegende statistische Maße abgelesen werden.

### Histogramme

In [ ]:


def truncate_label(label, length=5):
    return label[:length]

colors = sns.color_palette(None, 3)
columns = ["DayOfWeek", "PdDistrict", "Resolution"]


fig, axes = plt.subplots(4, 2, figsize = (10, 15), tight_layout=True)

axes[0, 0].hist(crime["Category"], bins=5, align="mid")

#Tag der Woche als Zahl für Sortierung der Balken
axes[0, 1].hist(crime['Dates'].dt.weekday + 1, bins=7, align="mid")

axes[1, 0].hist(crime["PdDistrict"], bins=len(crime["PdDistrict"].unique()))
axes[1, 1].hist(crime["Resolution"], bins=len(crime["Resolution"].unique()))
axes[2, 0].hist(crime["Year"].sort_values(), bins=13)
axes[2, 1].hist(crime["Month"].sort_values(), bins=12)
axes[3, 0].hist(crime["X"], bins=50)
axes[3, 1].hist(crime["Y"], bins=50)

# Rotate x-axis tick labels
for ax in axes.flatten():
    ax.tick_params(axis='x', rotation=45)
    ax.xaxis.set_ticks_position('bottom')  # Place ticks at the bottom for better visibility
    ax.set_xticklabels([truncate_label(label.get_text()) for label in ax.get_xticklabels()])

plt.tight_layout()

plt.show()

Hier werden Histogramme für verschiedene Merkmale des Kriminalitätsdatensatzes erstellt, einschließlich der Kategorie der Straftaten, dem Wochentag, dem Polizeibezirk, der Auflösung, dem Jahr, dem Monat sowie den geografischen Koordinaten X und Y. 

### Zeitliche Regression

In [ ]:
sns.set_style("ticks")
sns.set_context('notebook', font_scale = 1)

crime_year_cat = crime.groupby(by=["Year", "Category"], as_index=False).count()
data = crime_year_cat.pivot_table(index="Year", columns="Category", values="Descript")

fig = plt.figure(figsize=(16,9))

for category in categories:
    plt.plot(data[category], label=category)

plt.legend(loc="upper left")
tix = plt.xticks()[0]
plt.xticks(tix, rotation=90, ha="center")
plt.show()


Es folgt eine Visualising der zeitlichen Regression und Entwicklung der Straftaten in verschiedenen Kategorien über die Jahre gruppiert nach 'Year', wodurch Trends und Muster im zeitlichen Verlauf sichtbar werden. Besonders Auffällig ist der Abfall von 'Larceny/Theft' nach dem Jahr 2014. 

In [ ]:
crime_yearmonth_cat = crime.groupby(by=["YearMonth", "Category"], as_index=False).count()
data = crime_yearmonth_cat.pivot_table(index="YearMonth", columns="Category", values="Descript")

fig = plt.figure(figsize=(16,9))

for category in categories:
    plt.plot(data[category], label=category)

plt.legend(loc="upper left")
tix = plt.xticks()[0]
plt.xticks(tix[::6], rotation=90, ha="center")
plt.show()

Diese Visualisierung zeigt ebenfalls die zeitliche Entwicklung der Straftaten nach den jeweiligen Kategorien. In diesem Fall wird jedoch eine Gruppierung nach 'YearMonth' vorgenommen, damit eine feinere zeitliche Auflösung ersichtlich wird. 

### Kartogramme

In [ ]:

sf_df = gpd.read_file("data/SF Find Neighborhoods.geojson").to_crs({'init': 'epsg:4326'})

geometry = [Point(xy) for xy in zip(crime.X, crime.Y)]
crime_gdf = GeoDataFrame(crime, crs="EPSG:4326", geometry=geometry)


point = crime_gdf
poly  = sf_df

pointInPolys = sjoin(point, poly, how='left')
pointInPolys.drop_duplicates(subset=['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
                                     'Resolution', 'Address', 'X', 'Y', 'YearMonth', 'Year', 'Month', 'Day',
                                     'TimeOfDay'], inplace=True)

pointInPolys = pointInPolys.rename(columns ={"name" : "crdistrict"})

crime = crime.join(pointInPolys[["CrimeId", "crdistrict"]], on="CrimeId", lsuffix="_")

grouped = pointInPolys.groupby('index_right').count()


Hier wird eine GeoDataFrame (sf_df) aus einer GeoJSON-Datei eingelesen, die die Grenzen der Stadtviertel von San Francisco enthält. Die to_crs-Methode wird verwendet, um die Koordinatenreferenz des GeoDataFrames auf das Standardformat (EPSG:4326) zu ändern, das Längen- und Breitengrade verwendet. 
Vgl. City and County of San Francisco. (2016). SF Find Neighborhoods. [Dataset]. Socrata. https://data.sfgov.org/Geographic-Locations-and-Boundaries/SF-Find-Neighborhoods/pty2-tcw4

Insgesamt ermöglicht dieses Vorgehen die Zuordnung von Kriminalitätsdaten zu den entsprechenden Stadtvierteln von San Francisco und die Berechnung der Anzahl der Vorkommen in jedem Viertel. Dies ist nützlich, um räumliche Muster und Hotspots von Kriminalität zu identifizieren.

In [ ]:
heat_districts = pointInPolys.groupby(['index_right', 'crdistrict'])["Dates"].count()
heat_districts

In heat_districts werden dafür Indexwerte für die einzelnen Stadtviertel erstellt. Die Werte repräsentieren dabei die Anzahl der Kriminalitätsvorfälle in jedem Stadtviertel.

In [ ]:

geom = sf_df.pop('geometry')
sf_df = sf_df.join(geom, how='inner')
sf_df["polygons"] = ""
for i,row in sf_df.iterrows():
    geometry = row["geometry"]
    if geometry.geom_type == 'MultiPolygon':
        polygons = []
        for polygon in geometry.geoms:
            exterior_coords = list(polygon.exterior.coords)
            interior_coords = [list(interior.coords) for interior in polygon.interiors]
            polygons.append(Polygon(exterior_coords, interior_coords))
    else:
        polygons = [Polygon(list(geometry.exterior.coords))]
    sf_df.at[i, "polygons"] = polygons

sf_df["polygons"] = sf_df["polygons"].explode()

sf_df["index_right"] = sf_df.index
sf_df["heat"] = sf_df.merge(heat_districts, on="index_right")["Dates"]
sf_df["geometry"] = sf_df["polygons"]

gdf = gpd.GeoDataFrame(sf_df)




In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10, 20), layout='constrained')

gdf.plot(ax=ax[0], alpha=1, edgecolor='k', linewidth=0.5, column="heat", cmap="PuRd")
cbar = plt.colorbar(ax[0].get_children()[0], ax=ax[0], orientation="horizontal", shrink=0.6)
cbar.set_label("Total Amount of Crimes")

sf_df = gpd.read_file("data/SF Find Neighborhoods.geojson").to_crs({'init': 'epsg:4326'})
sf_df.plot(ax=ax[1], alpha=0.2, edgecolor='k', linewidth=0.5, zorder=2)
ax[1].scatter(data=crime.loc[crime["Y"]<80], x="X", y="Y",alpha=0.2, color="pink", zorder=1)

ax[1].set_xlim(ax[0].get_xlim())
ax[1].set_ylim(ax[0].get_ylim())
ax[0].set_aspect('equal', adjustable='box')
ax[1].set_aspect('equal', adjustable='box')

sns.despine(ax=ax[0], left=True, right=True, top=True, bottom=True)
sns.despine(ax=ax[1], left=True, right=True, top=True, bottom=True)

ax[0].xaxis.set_ticks([])
ax[0].yaxis.set_ticks([])

ax[1].xaxis.set_ticks([])
ax[1].yaxis.set_ticks([])    
    
plt.show()



Insgesamt kann in der Heat-Map ein deutliches Muster erkannt werden. Die Kriminalfälle bündeln sich besonders in den Stadtvierteln im Nord-Osten von San Francisco.

## Feature Engineering

An dieser Stelle werden neue Spalten generiert, die den Datensatz bereichern und den Modellen helfen können.

### Feiertage

Zuerst werden die US-Feiertage importiert und dem Datensatz hinzugefügt.

In [ ]:
import holidays

usa_holidays = pd.Series(holidays.country_holidays('US',  years=range(crime["Dates"].dt.year.min(), crime["Dates"].dt.year.max())))

usa_holidays = pd.DataFrame(usa_holidays)
usa_holidays.columns = ["Holiday"]
usa_holidays["YearMonthDay"] = pd.to_datetime(usa_holidays.index)
usa_holidays.reset_index(drop=True)

crime["YearMonthDay"] = pd.to_datetime(crime["YearMonthDay"])
crime = crime.merge(usa_holidays, on="YearMonthDay", how="left")

crime["Holiday"].fillna("None", inplace=True)

In [ ]:
crime.head()

In [ ]:
df_district_holiday = pd.crosstab(crime['Category'], crime['Holiday'])
df_district_holiday = df_district_holiday.loc[:, df_district_holiday.columns != 'None']
df_district_holiday.head()

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df_district_holiday, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Category vs Holiday')
plt.show()

In dieser Heat-Map werden die Feiertage den Kriminalkategorien entgegengestellt. Hier lässt sich ablesen, dass es die meisten Verbechen am Labor Day und Washingtons Birthday gibt.

### Wetter

Die Daten werden um ein weiteres Feature erweitert. Heirfür werden die Wetterdaten für die entsprechende Region importiert.

In [ ]:
crime.head()

In [ ]:
crime["hours"] = pd.to_datetime(crime['Dates'].dt.strftime('%Y-%m-%d %H'))

In [ ]:
from meteostat import Point, Hourly
import pandas as pd
import matplotlib.pyplot as plt

start_date = crime['Dates'].min()
end_date = crime['Dates'].max()

san_francisco = Point(37.7749, -122.4194, 10)

# Get hourly data for the specified date range
sf_weather = pd.DataFrame(Hourly(san_francisco, start_date, end_date).fetch())

sf_weather["hours"] = pd.to_datetime(sf_weather.index.strftime('%Y-%m-%d %H'))
crime = crime.merge(sf_weather, how="left", on="hours")

In [ ]:
crime.describe()

In [ ]:
temp_cat_ct = pd.crosstab(crime["temp"], crime["Category"],  normalize='index')
temp_cat_ct.plot(kind="area",  stacked="true", figsize=(12, 8))

### Straßen

Außerdem werden Straßen dem Datensatz hinzugefügt. Ziel ist es eine Übersicht der *gefährlichsten* Straßen in San Francisco zu bekommen.

In [ ]:
is_block = crime["Address"].str.contains(" /")
crime.loc[is_block, "Street/Block"] = crime.loc[is_block, "Address"].copy()
crime.loc[is_block, "Street_1"] = crime.loc[is_block, "Address"].apply(lambda x: x.split(" /")[0]).copy()
crime.loc[is_block, "Street_2"] = crime.loc[is_block, "Address"].apply(lambda x: x.split(" /")[1]).copy()

is_street = crime["Address"].str.contains(" of ")
crime.loc[is_street, "Street/Block"] = crime.loc[is_street, "Address"].apply(lambda x: x.split(" of ")[1]).copy()
crime.loc[is_street, "Street_1"] = crime.loc[is_street, "Address"].apply(lambda x: x.split(" of ")[1]).copy()
crime.loc[is_street, "Street_2"] = None
crime.head()

In [ ]:
str_cat_ct = pd.crosstab(crime["Street/Block"], crime["Category"], margins=True, margins_name="Total")

str_cat_ct = str_cat_ct[str_cat_ct.index != "Total"]
top_ten_str = str_cat_ct.sort_values(by="Total", ascending=False).head(10)
top_ten_str

Anhand der Tabelle wird deutlich, dass bei den Top drei der gefährlichsten Straßen mit den meisten Kriminalfällen um die Market St, Mission St und Bryant St handelt. Hierbei ist jedoch die Länge der Straße zu beachten, damit diese Werte in ein geeignetes Verhältnis gesetzt werden können. Einige Straßen reichen durch nahezu die komplette Stadt, während andere kurz bemessen sind.

Zuletzt folgt die Visualisierung der Straßen anhand von einem Balkendiagramm.

In [ ]:
top_ten_str.drop('Total', axis=1).plot(kind='bar', stacked=True, figsize=(12, 8))

### Korrelation

Nachfolgend werden weitere Korrelationen berechent und visualisiert, um erneut Muster und Zusammenhänge in den Daten zu erkennen.

#### Korrelation zwischen Kategorien und Bezirken

Dafür wird im ersten Schritt eine Kreuztabelle für den Bezirk und die Kategorie erstellt.

In [ ]:
df_district_cat = pd.crosstab(crime['Category'], crime['PdDistrict'])
df_district_cat.head()

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df_district_cat, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Category vs District')
plt.show()

Anhand dieser Heat-Map wird ersichtlich, dass es besonders im Bezitk 'Southern' ein hohes Aufkommen von 'Larceny/Theft'-Kriminalfällen gibt.

#### Korrelation zwischen den kategorischen Spalten

Für die weitere Betrachtung wird die Korrelation zwischen den kategorischen Spalten betrachtet.

In [ ]:
unique_val_col = pd.DataFrame(crime.nunique())
corr_columns = list(unique_val_col.loc[(unique_val_col[0] > 0) & (unique_val_col[0] <= 2500)].index)
print(corr_columns)

In [ ]:

x = [i[0] for i in product(corr_columns, corr_columns)]
y = [i[1] for i in product(corr_columns, corr_columns)]


corr_df = pd.DataFrame(index=corr_columns, columns=corr_columns)

for i in range(0,len(x)):
    if x[i] == y[i]:
        corr_df.loc[x[i], y[i]] = 1
        corr_df.loc[y[i], x[i]] = 1
    if pd.isnull(corr_df.loc[x[i], y[i]]):
        temp_ct = pd.crosstab(crime[x[i]], crime[y[i]])

        X2 = stats.chi2_contingency (temp_ct, correction= False )[0]
        n = sum(temp_ct.sum())
        minDim = min( temp_ct.shape )-1

        V = np.sqrt((X2/n) / minDim)

        corr_df.loc[x[i], y[i]] = V
        corr_df.loc[y[i], x[i]] = corr_df.loc[x[i], y[i]]

Hierfür wurde die Berechnung von Cramers-V genutzt.

Vgl. Cramér, H. (1946). Mathematical Methods of Statistics. Princeton: Princeton University Press, p. 282 (Chapter 21. The two-dimensional case). ISBN 0-691-08004-6.

In [ ]:
corr_df = corr_df.apply(pd.to_numeric, errors='coerce')

mask = np.zeros_like(corr_df)
mask[np.triu_indices_from(mask, k=1)] = True

plt.figure(figsize=(14, 12))
sns.heatmap(corr_df, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, mask=mask)
plt.title('Korrelations Matrix ')
plt.show()

In [ ]:
__corr_df = corr_df.loc[corr_df["Category"] >= 0.09]
#__corr_df = __corr_df.drop(["YearMonth", "Year", "Street_2"])
__corr_df = __corr_df.apply(pd.to_numeric, errors='coerce')
__corr_df = __corr_df[__corr_df.index]

mask = np.zeros_like(__corr_df)
mask[np.triu_indices_from(mask, k=1)] = True

plt.figure(figsize=(10, 8))
sns.heatmap(__corr_df, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, mask=mask)
plt.title('Korrelations Matrix ')
plt.show()

Insgesamt ermöglicht diese Heatmap eine visuelle Darstellung der Korrelationsmatrix. Durch die Farben und die annotierten Werte können leicht Muster und Stärke der Korrelationen zwischen den verschiedenen Variablen im Datensatz erkannt werden.

# 6. Vorbereitung

Anhand der Korrelation können wir ablesen, welche Spalten den größten Einfluss auf die Kategorie hat, welche wir letztendlich vorhersagen wollen.

In [ ]:
__corr_df["Category"]


Im nächsten Schritt werden die Spalten ausgewählt, welche mindestens eine Korrelation von 0.05 haben


Da Year und YearMonth fast die gleiche Korrelation mit Category haben und untereinander eine sehr hohe Korrelation haben, 
beschränken wir uns auf YearMonth und filtern Year raus. 

<!-- Resolution ist leider nicht in den Test-Daten vorhanden, daher nützt es nichts die Modelle mit dieser Spalte zu trainieren und sie wird ebenfalls entfernt.  -->

Und da wir Category vorhersagen sollen, wird es auch im Training nicht berücksichtigt

In [ ]:
onehot_cols = __corr_df.index


#Obwohl Resolution für die Kaggle-Challange nicht verwendet werden darf, darf es im Rahmen des Projekts zur Klassifikation dienen.
#onehot_cols.remove("Resolution")


onehot_cols = onehot_cols.drop(["Resolution", "Category", "Descript", "YearMonth", 'YearMonthDay', "Year", "Street_2", "Street_1"])
onehot_cols

## One-Hot-Encoding

In [ ]:
crime_oh = pd.get_dummies(data=crime, columns=onehot_cols, dtype=float, prefix=["_" + i for i in onehot_cols])
crime_oh = crime_oh.drop(["CrimeId_", "Street_2", "Street_1"], axis=1)
crime_oh["Category"] = crime["Category"]
crime_oh.head()

## Daten Angleichen

Die Kategorien sind momentan unausgeglichen, was letztendlich zu einem Bias im Modell führen kann. 
Hierfür wurde SMOTE in Kombination mit Undersampling verwendet, also die Daten werden zu erst auf basis der vorhandenen Daten hochskaliert und dann werden zufällige Werte gleichermaßen gelöscht. 
Diese Kombination funktioniert besser als reines Undersampling (Vgl. Chawla, N. V., Bowyer, K. W., Hall, L. O. & Kegelmeyer, W. P. (2002). SMOTE: Synthetic Minority Over-sampling technique. Journal Of Artificial Intelligence Research, 16, 321–357. https://doi.org/10.1613/jair.953)

In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler



X, y = crime_oh.filter(like='_', axis=1), crime_oh["Category"]
print("Shape vor Angleichung",X.shape)
class_size = round(X.shape[0] / 5 * 0.7)

oversampling = SMOTE()
undersampling = RandomUnderSampler(sampling_strategy={i: class_size for i in categories})
# steps = [('SMOTE', oversampling), ('RandomUnderSampler', undersampling)]
steps = [('SMOTE', oversampling)]
pipeline = Pipeline(steps=steps)

X, y = pipeline.fit_resample(X, y)
print("Shape nach Angleichung",X.shape)

In [ ]:
for i in range(0,10,1)[0:20]

# 7. Modellierung

## Entscheidungsbaum

### Daten Filtern

Um die Modelle und Rechenleistung zu verbessern, müssen die Spalten, die den Modellen übergeben werden (Feautures) reduziert werden.
Mit Hilfe des ChiQuadrat Tests können die relevantesten Spalten gefunden werden.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
feature_selector = SelectKBest(chi2, k=100).fit(X, y)
X_new = X.iloc[:,feature_selector.get_support(indices=True)]

Kreuzvalidierung eines einfachen Entscheidungsbaumes mit den 100 besten Spalten

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

cross_validate(DecisionTreeClassifier(), X, y, cv=5)["test_score"]

Kreuzvalidierung eines einfachen Entscheidungsbaumes mit allen Spalten

In [ ]:
cross_validate(DecisionTreeClassifier(), X, y, cv=5)["test_score"]

Es wird deutlich, dass alle Feautures nicht zwangsweise das Modell verbessern, also gilt es herauszufinden, welche Anzahl die beste ist.

In [ ]:
# decision_tree = DecisionTreeClassifier(
#     criterion         = 'entropy',
#     max_depth         = 16,
#     max_leaf_nodes    = 80,
#     min_samples_leaf  = 1,
#     min_samples_split = 500
# )

# decision_tree = DecisionTreeClassifier()
# 
# scores = []
# k=2
# while k < X.shape[1]/2:
#     feature_selector = SelectKBest(chi2, k=k).fit(X, y)
#     X_temp = X.iloc[:,feature_selector.get_support(indices=True)]
#     cv_results = cross_validate(decision_tree, X, y, cv=5)
#     print("k\t\t=\t",k)
#     print("score\t=\t",cv_results['test_score'].mean())
#     print("-------------------------------------------------")
#     scores = np.append(scores, (k, cv_results['test_score'].mean()))
#     k=k*2
    

In [ ]:
scores_df = pd.DataFrame.from_records([(scores[i],scores[i+1]) for i in range(0,len(scores),2)],  columns=["k", "score"])
sns.lineplot(scores_df, x="k", y="score")

Ein Entscheidungsbaum mit Standardeinstellungen lieferten die besten Werte bei einer Spaltenanzahl zwischen 32 und 128.

Nun kann das Prozedere wiederholt werden mit kleineren Schritten.

In [ ]:
decision_tree = DecisionTreeClassifier()

scores = []
for k in range(30,130,10):
    feature_selector = SelectKBest(chi2, k=k).fit(X, y)
    X_temp = X.iloc[:,feature_selector.get_support(indices=True)]
    cv_results = cross_validate(decision_tree, X_temp, y, cv=5)
    print("k\t\t=\t",k)
    print("score\t=\t",cv_results['test_score'].mean())
    print("-------------------------------------------------")
    scores = np.append(scores, (k, cv_results['test_score'].mean()))

In [ ]:
scores_df_2 = pd.DataFrame.from_records([(scores[i],scores[i+1]) for i in range(0,len(scores),2)],  columns=["k", "score"])
sns.lineplot(scores_df_2, x="k", y="score")

Der beste Score liegt bei 60.

In [ ]:
feature_selector = SelectKBest(chi2, k=1000).fit(X, y)
X = X.iloc[:,feature_selector.get_support(indices=True)]

In [ ]:
decision_tree = DecisionTreeClassifier()

decision_tree_cv = cross_validate(decision_tree, X_new, y, cv=5)
print("Ergebnisse der einzelnen Folds: ",decision_tree_cv['test_score'])
print("Durchschnittlicher Score: ",decision_tree_cv['test_score'].mean())

Nun gilt es die besten Parameter zu finden, dazu werden zwei Werkzeuge benutzt: RandomizedSearchCV und GridSearchCV.
Beide Methoden verwenden Kreuzvalidierung und generieren Scores für Modelle mit verschiedenen Parametern.

### Hyperparameter Tuning

#### RandomizedSearchCV



In [ ]:
from sklearn.model_selection import RandomizedSearchCV

decision_tree = DecisionTreeClassifier()

param_grid = {
    "criterion": ['entropy', 'gini'],
    "max_depth" : range(1,51,1),
    "max_leaf_nodes": range(10,110,10),
    "min_samples_leaf" : range(50, 1000,50),
    "min_samples_split" : range(50,1000,50),
}



random_search_cv = RandomizedSearchCV(
decision_tree, 
    param_distributions=param_grid, 
    n_iter=50, 
    verbose=5
    
)

random_search_cv = random_search_cv.fit(X, y)

In [ ]:
# random_search_results = pd.read_csv("data/random_search.csv")
random_search_results = pd.DataFrame(random_search_cv.cv_results_)

params_column = random_search_results["params"]
params_df = params_column.apply(lambda x: pd.Series(eval(str(x))))
random_search_results = pd.concat([random_search_results, params_df], axis=1)
random_search_results = random_search_results.drop(["params"], axis=1)

random_search_results

In [ ]:
rs_top_10 = random_search_results.loc[random_search_results["rank_test_score"] <=10]
rs_top_10 = rs_top_10.filter(like="param_").iloc[:,0:4].astype('int64').describe()

In [ ]:
rs_top_10

In [ ]:
random_search_cv.best_params_

Wenn man die Top 10 Parameter Konfigurationen betrachtet, lassen sich die Parameter weiter eingrenzen.

In [ ]:
param_grid = {
    "criterion": ['gini'],
    "min_samples_split" : range(int(rs_top_10.loc["min","param_min_samples_split"]),int(rs_top_10.loc["max","param_min_samples_split"]),200),
    "min_samples_leaf" : range(int(rs_top_10.loc["min","param_min_samples_leaf"]),int(rs_top_10.loc["max","param_min_samples_leaf"]),100),
    "max_leaf_nodes": range(int(rs_top_10.loc["min","param_max_leaf_nodes"]),int(rs_top_10.loc["max","param_max_leaf_nodes"]),20),
    "max_depth" : range(int(rs_top_10.loc["min","param_max_depth"]),int(rs_top_10.loc["max","param_max_depth"]),10)
}

print(param_grid)

#### GridSearchCV

In [ ]:
from joblib import parallel_backend

with parallel_backend('threading', n_jobs=-1):
    from sklearn.model_selection import GridSearchCV
    grid_search_cv = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid, cv= 5,verbose=5, n_jobs=-1)
    grid_search_cv.fit(X, y)


In [ ]:
# grid_search_results = pd.read_csv("data/grid_search.csv")
grid_search_results = pd.DataFrame(grid_search_cv.cv_results_)

params_column = grid_search_results["params"]
params_df = params_column.apply(lambda x: pd.Series(eval(str(x))))
grid_search_results = pd.concat([grid_search_results, params_df], axis=1)
grid_search_results = grid_search_results.drop(["params"], axis=1)

grid_search_results

In [ ]:
grid_search_results.loc[grid_search_results["rank_test_score"] ==1].filter(like="param_")

In [ ]:
print(grid_search_results.best_params_,grid_search_results.best_score_)

In [ ]:
random_search_results.to_csv("random_search_tree.csv")
grid_search_results.to_csv("grid_search_tree.csv")

### Optimierter Baum

Nun kann der finale Baum mit den Parametern generiert werden.

In [ ]:
decision_tree = DecisionTreeClassifier(
    criterion         = 'entropy',
    max_depth         = 34,
    max_leaf_nodes    = 100,
    min_samples_leaf  = 500,
    min_samples_split = 450
)

cv_tree = cross_validate(decision_tree, X, y, cv=5)
cv_tree["test_score"]

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_pred = cross_val_predict(decision_tree, X, y, cv=5)

conf_matrix = confusion_matrix(y, y_pred, labels=categories)

#Konfusionsmatrix anzeigen (optional mit Seaborn für eine bessere Darstellung)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix with k-Fold Cross-Validation for Decision Tree')
plt.show()

## Support Vector Machines

### Daten Filtern

Es gilt herauszufinden, welche Anzahl an Spalten die optimalste ist.

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC()

scores = []
k=2
while k < X.shape[1]/2:
    feature_selector = SelectKBest(chi2, k=k).fit(X, y)
    X_temp = X.iloc[:,feature_selector.get_support(indices=True)]
    cv_results = cross_validate(svm, X, y, cv=5)
    print("k\t\t=\t",k)
    print("score\t=\t",cv_results['test_score'].mean())
    print("-------------------------------------------------")
    scores = np.append(scores, (k, cv_results['test_score'].mean()))
    k=k*2
    

In [ ]:
scores_df = pd.DataFrame.from_records([(scores[i],scores[i+1]) for i in range(0,len(scores),2)],  columns=["k", "score"])
sns.lineplot(scores_df, x="k", y="score")

Ein Entscheidungsbaum mit Standardeinstellungen lieferten die besten Werte bei einer Spaltenanzahl zwischen 32 und 128.

Nun kann das Prozedere wiederholt werden mit kleineren Schritten.

In [ ]:
svm = LinearSVC()

scores = []
for k in range(30,130,10):
    feature_selector = SelectKBest(chi2, k=k).fit(X, y)
    X_temp = X.iloc[:,feature_selector.get_support(indices=True)]
    cv_results = cross_validate(svm, X_temp, y, cv=5)
    print("k\t\t=\t",k)
    print("score\t=\t",cv_results['test_score'].mean())
    print("-------------------------------------------------")
    scores = np.append(scores, (k, cv_results['test_score'].mean()))

In [ ]:
scores_df_2 = pd.DataFrame.from_records([(scores[i],scores[i+1]) for i in range(0,len(scores),2)],  columns=["k", "score"])
sns.lineplot(scores_df_2, x="k", y="score")

Der beste Score liegt bei 60.

In [ ]:
feature_selector = SelectKBest(chi2, k=1000).fit(X, y)
X = X.iloc[:,feature_selector.get_support(indices=True)]

In [ ]:
#Mit Einstellungen
#0.5579648513453019

#Ohne Einstellungen
#0.5663729942716096

svm = LinearSVC()

decision_tree_cv = cross_validate(decision_tree, X_new, y, cv=5)
print("Ergebnisse der einzelnen Folds: ",decision_tree_cv['test_score'])
print("Durchschnittlicher Score: ",decision_tree_cv['test_score'].mean())

Nun gilt es die besten Parameter zu finden, dazu werden zwei Werkzeuge benutzt: RandomizedSearchCV und GridSearchCV.
Beide Methoden verwenden Kreuzvalidierung und generieren Scores für Modelle mit verschiedenen Parametern.

### Hyperparameter Tuning

#### GridSearchCV

In [ ]:
from joblib import parallel_backend

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'loss': ['hinge', 'squared_hinge'],
    'penalty': ['l1', 'l2'],
    'dual': [True, False],
    'tol': [1e-3, 1e-4, 1e-5]
}

with parallel_backend('threading', n_jobs=-1):
    from sklearn.model_selection import GridSearchCV
    grid_search_cv = GridSearchCV(estimator=svm, param_grid=param_grid, cv= 5,verbose=5, n_jobs=-1)
    grid_search_cv.fit(X, y)


In [ ]:
# grid_search_results = pd.read_csv("data/grid_search.csv")
grid_search_results = pd.DataFrame(grid_search_cv.cv_results_)

params_column = grid_search_results["params"]
params_df = params_column.apply(lambda x: pd.Series(eval(str(x))))
grid_search_results = pd.concat([grid_search_results, params_df], axis=1)
grid_search_results = grid_search_results.drop(["params"], axis=1)

grid_search_results

In [ ]:
grid_search_results.loc[grid_search_results["rank_test_score"] ==1].filter(like="param_")

In [ ]:
print(grid_search_results.best_params_,grid_search_results.best_score_)

In [ ]:
grid_search_results.to_csv("grid_search_svm.csv")

### Optimierte Support Vector Machine

Nun kann das finale Modell mit den Parametern generiert werden.

In [ ]:
svm = LinearSVC(
    C = 0.1,
    loss = 'hinge',
    penalty = 'l2',
    dual = True,
    tol = 1e-4
)

cross_validate(svm, X, y, cv=5)["test_score"]

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_pred = cross_val_predict(svm, X, y, cv=5)

conf_matrix = confusion_matrix(y, y_pred, labels=categories)

#Konfusionsmatrix anzeigen (optional mit Seaborn für eine bessere Darstellung)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix with k-Fold Cross-Validation for Decision Tree')
plt.show()